## Bert用于句子相似度的应用步骤
在句子相似度任务中，主要目标是通过模型对两个句子的相似度进行建模，并且输出一个连续的相似性分数，通常是0到5之间，数值越大表示句子越相似。

1. 任务定义
* 任务是给定两个句子，模型输出它们的相似度分数。
2. 数据准备
* 为了训练模型，通常需要一个包含句子对和相应相似度分数的数据集。句子对会作为输入，目标是让模型预测的分数接近数据集中标注的相似度分数。
3. 模型选择
* 我们使用预训练的Bert模型，它已经在大量文本上进行了训练，可以很好的处理自然语言理解任务。Bert可以将两个句子作为输入，并通过模型进行处理，提取它们的语义。
4. 输入处理
* Bert模型需要将两个句子拼接在一起作为输入，并且在它们中间用特殊标记`[SET]`分开。
* 输入的形式通常是：`[CLS]句子1[SEP]句子2[SEP]`。其中`[CLS]`是一个特殊的分类标记，表示整个句子的语义摘要；而`[SEP]`用来分隔句子
5. 模型输出
* BERT会对输入的句子进行编码，并生成一个表示句子对相似性的向量。然后，我们在Bert输出的基础上添加一个回归层。回归层的作用是将这个向量转换成一个相似度分数。
6. 损失函数
* 我们会计算模型预测的相似度分数与数据集中真实标注分数之间的差异。我们使用均方误差（MSE）作为函数。通过反向传播和优化起，模型会不断调整参数，使得损失值减小，也就是模型的预测越来越准确。
7. 训练过程
8. 评估模型
* 在验证和测试阶段，我们通过将模型预测的相似度分数和真实分数进行比较来评估模型性能。通常也是通过均方误差来衡量模型的准确性。均方误差越小，模型的预测就越准确。

In [15]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.metrics import mean_squared_error
import numpy as np

### 1. 加载Hugging Face数据集

In [3]:
# 加载 STS-B 数据集
dataset = load_dataset("glue", "stsb")
# sts-b是一个句子相似度的基准任务

In [17]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})


In [19]:
df_train=dataset['train'].to_pandas()
print(df_train.head())

                                       sentence1  \
0                         A plane is taking off.   
1                A man is playing a large flute.   
2  A man is spreading shreded cheese on a pizza.   
3                   Three men are playing chess.   
4                    A man is playing the cello.   

                                           sentence2  label  idx  
0                        An air plane is taking off.   5.00    0  
1                          A man is playing a flute.   3.80    1  
2  A man is spreading shredded cheese on an uncoo...   3.80    2  
3                         Two men are playing chess.   2.60    3  
4                 A man seated is playing the cello.   4.25    4  


In [20]:
df_test=dataset['test'].to_pandas()
print(df_test.head())

                                       sentence1  \
0                    A girl is styling her hair.   
1       A group of men play soccer on the beach.   
2  One woman is measuring another woman's ankle.   
3                A man is cutting up a cucumber.   
4                       A man is playing a harp.   

                                          sentence2  label  idx  
0                      A girl is brushing her hair.   -1.0    0  
1  A group of boys are playing soccer on the beach.   -1.0    1  
2           A woman measures another woman's ankle.   -1.0    2  
3                      A man is slicing a cucumber.   -1.0    3  
4                      A man is playing a keyboard.   -1.0    4  


### 2. 加载Bert Tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

### 3. 定义数据集的预处理函数

In [12]:
def tokenize_function(examples):
    """
        使用Bert的tokenizer对句子进行编码，生成input_ids和attention_mask
    """
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length", max_length=128)

### 4. 数据集预处理

In [6]:
# 对训练集和验证集进行tokenization
# map方法将tokenize_function应用于训练和验证数据集
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 设置数据集的格式为 PyTorch tensors
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

### 5. 准备DataLoader

In [7]:
# 创建训练集和验证集 DataLoader
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=16, shuffle=True)
valid_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=16)

### 6. 模型构建

In [8]:
# 加载预训练的BERT模型，指定类别数为1，因为STS-B是回归任务
# num_labels=1表示这是一个单维度回归任务（句子相似度得分在0到5之间）
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 使用Adam优化器
optimizer = AdamW(model.parameters(), lr=2e-5)

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 7. 模型训练

In [10]:
# 定义训练过程
def train(model, dataloader, optimizer, num_epochs=3):
    model.train()
    loss_fn = torch.nn.MSELoss()  # 回归任务的损失函数使用均方误差
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label'].unsqueeze(1)  # 调整label形状
            
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")


In [11]:
# 开始训练
train(model, train_dataloader, optimizer, num_epochs=3)


Epoch 1/3, Loss: 1.30950252380636
Epoch 2/3, Loss: 0.4926092455370559
Epoch 3/3, Loss: 0.32483585472736093


### 7. 模型评估

In [13]:
def evaluate(model, dataloader):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            label = batch['label'].unsqueeze(1)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.append(outputs.logits.cpu().numpy())
            labels.append(label.cpu().numpy())
    
    # 将预测和真实值进行拼接
    preds = np.concatenate(preds).flatten()
    labels = np.concatenate(labels).flatten()
    
    # 计算均方误差（MSE）
    mse = mean_squared_error(labels, preds)
    print(f"Validation MSE: {mse:.4f}")

In [16]:
# 评估验证集上的表现
evaluate(model, valid_dataloader)

Validation MSE: 0.5731
